In [1]:
import numpy as np
import pandas as pd
import scipy, scipy.signal

from datetime import date
import time

import random
from random import seed
from random import random

import os, os.path
import shutil

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import matplotlib
import matplotlib.pyplot as plt
from pylab import imshow

import h5py
import sys


In [2]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/')
import NASA_core as nc
# import NASA_plot_core as rcp

In [3]:
from tslearn.metrics import dtw as dtw_metric

# https://dtaidistance.readthedocs.io/en/latest/usage/dtw.html
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis

# Read Fields Metadata

In [4]:
meta_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"
meta = pd.read_csv(meta_dir+"evaluation_set.csv")
meta_moreThan10Acr=meta[meta.ExctAcr>10]
print (meta.shape)
print (meta_moreThan10Acr.shape)
meta.head(2)

(6340, 8)
(3539, 8)


,ID,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,34.310305,2017/09/12,Grant
1,100204_WSDA_SF_2017,alfalfa hay,center pivot,wsda,62,61.826535,2017/08/09,Grant


In [5]:
print (len(meta.ID.unique()))
meta_lessThan10Acr=meta[meta.ExctAcr<10]
print (meta_lessThan10Acr.shape)

6340
(2801, 8)


# Read Training Set Labels

In [6]:
training_set_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data/"

ground_truth_labels = pd.read_csv(training_set_dir+"train_labels.csv")
print ("Unique Votes: ", ground_truth_labels.Vote.unique())
print (len(ground_truth_labels.ID.unique()))
ground_truth_labels.head(2)

Unique Votes:  [2 1]
1849


,ID,Vote
0,99837_WSDA_SF_2017,2
1,114615_WSDA_SF_2017,1


### Detect how many fields are less than 10 acres and report in the paper

In [7]:
print (len(meta[meta.ID.isin(list(ground_truth_labels.ID))].ID.unique()))
meta.head(2)

1849


,ID,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,34.310305,2017/09/12,Grant
1,100204_WSDA_SF_2017,alfalfa hay,center pivot,wsda,62,61.826535,2017/08/09,Grant


# Read the Data

In [13]:
VI_idx = "NDVI"
data_dir = "/Users/hn/Documents/01_research_data/NASA/VI_TS/04_regularized_TS/"

In [16]:
file_names = ["regular_Walla2015_"+ VI_idx + "_JFD.csv", 
              "regular_AdamBenton2016_"+ VI_idx + "_JFD.csv", 
              "regular_Grant2017_"+ VI_idx + "_JFD.csv", 
              "regular_FranklinYakima2018_"+ VI_idx + "_JFD.csv"]

data=pd.DataFrame()

for file in file_names:
    curr_file=pd.read_csv(data_dir + file)
    curr_file['human_system_start_time'] = pd.to_datetime(curr_file['human_system_start_time'])
    
    # These data are for 3 years. The middle one is the correct one
    all_years = sorted(curr_file.human_system_start_time.dt.year.unique())
    if len(all_years)==3 or len(all_years)==2:
        proper_year = all_years[1]
    elif len(all_years)==1:
        proper_year = all_years[0]

    curr_file = curr_file[curr_file.human_system_start_time.dt.year==proper_year]
    data=pd.concat([data, curr_file])

data.reset_index(drop=True, inplace=True)
data.head(2)

,ID,human_system_start_time,NDVI
0,135073_WSDA_SF_2015,2015-01-10,0.163569
1,135073_WSDA_SF_2015,2015-01-20,0.028382


In [17]:
ground_truth = data[data.ID.isin(list(ground_truth_labels.ID.unique()))].copy()
len(ground_truth.ID.unique())

1849

# Toss small fields

In [18]:
len(meta_moreThan10Acr.ID.unique())

3539

In [19]:
ground_truth_labels_extended = pd.merge(ground_truth_labels, meta, on=['ID'], how='left')
ground_truth_labels = ground_truth_labels_extended[ground_truth_labels_extended.ExctAcr>=10].copy()
ground_truth_labels.reset_index(drop=True, inplace=True)

print ("There are [{:.0f}] fields in total whose area adds up to [{:.2f}].".format(len(ground_truth_labels_extended), \
                                                                     ground_truth_labels_extended.ExctAcr.sum()))


print ("There are [{:.0f}] fields >10 acres whose area adds up to [{:.2f}].".format(len(ground_truth_labels), \
                                                                    ground_truth_labels.ExctAcr.sum()))


There are [1849] fields in total whose area adds up to [85573.68].
There are [1342] fields larger than 10 acres whose area adds up to [83430.79].


In [23]:
ground_truth_labels[ground_truth_labels.Vote==1].shape

(1091, 9)

In [24]:
ground_truth = ground_truth[ground_truth.ID.isin((list(meta_moreThan10Acr.ID)))].copy()
ground_truth_labels = ground_truth_labels[ground_truth_labels.ID.isin((list(meta_moreThan10Acr.ID)))].copy()

ground_truth.reset_index(drop=True, inplace=True)
ground_truth_labels.reset_index(drop=True, inplace=True)

# Sort the order of time-series and experts' labels identically

In [25]:
ground_truth.sort_values(by=["ID", 'human_system_start_time'], inplace=True)
ground_truth_labels.sort_values(by=["ID"], inplace=True)

ground_truth.reset_index(drop=True, inplace=True)
ground_truth_labels.reset_index(drop=True, inplace=True)

assert (len(ground_truth.ID.unique()) == len(ground_truth_labels.ID.unique()))

print (list(ground_truth.ID)[0])
print (list(ground_truth_labels.ID)[0])
print ("____________________________________")
print (list(ground_truth.ID)[-1])
print (list(ground_truth_labels.ID)[-1])
print ("____________________________________")
print (list(ground_truth.ID.unique())==list(ground_truth_labels.ID.unique()))

100048_WSDA_SF_2017
100048_WSDA_SF_2017
____________________________________
99909_WSDA_SF_2017
99909_WSDA_SF_2017
____________________________________
True


In [27]:
# mins = ground_truth.groupby("ID")[VI_idx].min()
# maxs = ground_truth.groupby("ID")[VI_idx].max()
# _ranges = maxs-mins
# _ranges = pd.DataFrame(_ranges)
# _ranges.reset_index(inplace=True)

# mins = pd.DataFrame(mins)
# mins.reset_index(inplace=True)


# _ranges.rename(columns = {'NDVI':'NDVI_range'}, inplace = True)
# mins.rename(columns = {'NDVI':'NDVI_min'}, inplace = True)

# _ranges.head(2)

# ground_truth = pd.merge(ground_truth, _ranges, on=['ID'], how='left')
# ground_truth = pd.merge(ground_truth, mins, on=['ID'], how='left')
# ground_truth["NDVI_ratio"] = (ground_truth["NDVI"]-ground_truth["NDVI_min"])/ground_truth["NDVI_range"]
# ground_truth.head(2)

# ID_1=ground_truth.ID.unique()[0]
# ID_2=ground_truth.ID.unique()[12]

# # _minimum = ground_truth[ground_truth.ID==ID_1].NDVI.values.min()
# # _range = ground_truth[ground_truth.ID==ID_1].NDVI.values.max()-_minimum
# # y_1 = (ground_truth[ground_truth.ID==ID_1].NDVI.values-_minimum)/_range
# # _minimum = ground_truth[ground_truth.ID==ID_2].NDVI.values.min()
# # _range = ground_truth[ground_truth.ID==ID_2].NDVI.values.max()-_minimum
# # y_2 = (ground_truth[ground_truth.ID==ID_2].NDVI.values-_minimum)/_range

# dtw_score = dtw_metric(ground_truth[ground_truth.ID==ID_1].NDVI.values, 
#                 ground_truth[ground_truth.ID==ID_2].NDVI.values)

# dtw_score_ratios = dtw_metric(ground_truth[ground_truth.ID==ID_1].NDVI_ratio.values, 
#                        ground_truth[ground_truth.ID==ID_2].NDVI_ratio.values)

# # print ("dtw score is {:.2f}.".format(dtw_score))
# plt.figure(figsize=(20,4))
# plt.subplot(1, 1, 1)

# # plot NDVIs
# plt.plot(range(len(ground_truth[ground_truth.ID==ID_1].human_system_start_time.values)),
#          ground_truth[ground_truth.ID==ID_1].NDVI.values,
#         c='k', linewidth=5);

# plt.plot(range(len(ground_truth[ground_truth.ID==ID_2].human_system_start_time.values)),
#          ground_truth[ground_truth.ID==ID_2].NDVI.values,
#         c='red', linewidth=5);

# # plot ratios
# plt.plot(range(len(ground_truth[ground_truth.ID==ID_1].human_system_start_time.values)), 
#          ground_truth[ground_truth.ID==ID_1].NDVI_ratio.values,
#          ".-.", c='k', linewidth=2, label="ratios");

# plt.plot(range(len(ground_truth[ground_truth.ID==ID_2].human_system_start_time.values)), 
#          ground_truth[ground_truth.ID==ID_2].NDVI_ratio.values,
#          ".-.", c='red', linewidth=2, label="ratios");

# title = "dtw score for NDVI is [{:.2f}] and for ratios is [{:.2f}].".format(dtw_score, dtw_score_ratios)
# plt.ylim([-0.2, 1.05]);
# plt.title(title , fontsize = 20);

# s1=ground_truth[ground_truth.ID==ID_1].NDVI_ratio.values
# s2=ground_truth[ground_truth.ID==ID_2].NDVI_ratio.values
# path = dtw.warping_path(s1, s2)
# dtwvis.plot_warping(s1, s2, path)
# distance = dtw.distance(s1, s2)

# d, paths = dtw.warping_paths(s1, s2, window=10, use_pruning=True);
# best_path = dtw.best_path(paths);
# dtwvis.plot_warpingpaths(s1, s2, paths, best_path);

# Widen Ground Truth Table

In [28]:
NDVI_colnames = [VI_idx + "_" + str(ii) for ii in range(1, 37) ]
columnNames = ["ID"] + NDVI_colnames
ground_truth_wide = pd.DataFrame(columns=columnNames, 
                                index=range(len(ground_truth.ID.unique())))
ground_truth_wide["ID"] = ground_truth.ID.unique()

for an_ID in ground_truth.ID.unique():
    curr_df = ground_truth[ground_truth.ID==an_ID]
    
    ground_truth_wide_indx = ground_truth_wide[ground_truth_wide.ID==an_ID].index
    ground_truth_wide.loc[ground_truth_wide_indx, "NDVI_1":"NDVI_36"] = curr_df.NDVI.values[:36]

In [29]:
print (len(ground_truth_wide.ID.unique()))
ground_truth_wide.head(2)

1342


,ID,NDVI_1,NDVI_2,NDVI_3,NDVI_4,NDVI_5,NDVI_6,NDVI_7,NDVI_8,NDVI_9,...,NDVI_27,NDVI_28,NDVI_29,NDVI_30,NDVI_31,NDVI_32,NDVI_33,NDVI_34,NDVI_35,NDVI_36
0,100048_WSDA_SF_2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.140785,0.157916,...,0.157398,0.176086,0.19269,0.142555,0.136615,0.130675,0.124734,0.118794,0.131257,0.14372
1,100081_WSDA_SF_2017,0.005831,0.003887,0.001944,0.0,0.131036,0.262071,0.419314,0.521772,0.656894,...,0.140417,0.197382,0.196319,0.186397,0.093199,0.0,0.093574,0.187148,0.144835,0.102522


# Split Train and Test Set

#### Make sure rows of ```ground_truth_allBands``` and ```ground_truth_labels``` are in the same order

In [30]:
ground_truth_labels.head(2)

,ID,Vote,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100048_WSDA_SF_2017,1,"bean, green",rill,wsda,18,18.033240,2017/05/14,Grant
1,100081_WSDA_SF_2017,1,wheat,rill,wsda,16,15.959744,2017/08/09,Grant


In [31]:
ground_truth_labels.CropTyp.unique()

array(['bean, green', 'wheat', 'onion', 'pea, green', 'corn, field',
       'corn, sweet', 'bean, dry', 'yellow mustard', 'potato', 'canola',
       'mint', 'grass seed', 'carrot', 'buckwheat', 'bluegrass seed',
       'grass hay', 'corn seed', 'pea, dry', 'pea seed', 'barley hay',
       'market crops', 'triticale', 'carrot seed', 'wheat fallow',
       'barley', 'alfalfa seed', 'oat hay', 'triticale hay'], dtype=object)

In [32]:
ground_truth.head(2)

,ID,human_system_start_time,NDVI
0,100048_WSDA_SF_2017,2017-01-06,0.0
1,100048_WSDA_SF_2017,2017-01-16,0.0


In [34]:
ground_truth_labels = ground_truth_labels.set_index('ID')
ground_truth_labels = ground_truth_labels.reindex(index=ground_truth_wide['ID'])
ground_truth_labels = ground_truth_labels.reset_index()

In [35]:
print (ground_truth_labels.ExctAcr.min())
ground_truth_labels.head(2)

10.0708703567


,ID,Vote,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100048_WSDA_SF_2017,1,"bean, green",rill,wsda,18,18.033240,2017/05/14,Grant
1,100081_WSDA_SF_2017,1,wheat,rill,wsda,16,15.959744,2017/08/09,Grant


In [36]:
ground_truth_labels=ground_truth_labels[["ID", "Vote"]]
ground_truth_labels.head(2)

,ID,Vote
0,100048_WSDA_SF_2017,1
1,100081_WSDA_SF_2017,1


In [37]:
x_train_df, x_test_df, y_train_df, y_test_df = train_test_split(ground_truth_wide, 
                                                                ground_truth_labels, 
                                                                test_size=0.2, 
                                                                random_state=0,
                                                                shuffle=True,
                                                                stratify=ground_truth_labels.Vote.values)

In [38]:
x_test_df.shape

(269, 37)

In [39]:
# out_name=training_set_dir+"train80_split_expertLabels_2Bconsistent.csv"
# y_train_df.to_csv(out_name, index = False)

# out_name=training_set_dir+"test20_split_expertLabels_2Bconsistent.csv"
# y_test_df.to_csv(out_name, index = False)

# Start SVM

# Definitions

  - **Precision** Of all instances we predict $\hat y = 1$, what fraction is actually 1.
     \begin{equation}\label{eq:precision}
        \text{Precision} = \frac{TP}{TP + FP}
     \end{equation}

  - **Recall** Of all instances that are actually $y = 1$, what fraction we predict 1.
     \begin{equation}\label{eq:recall}
         \text{Recall} = \text{TPR} = \frac{TP}{TP + FN}
     \end{equation}
     
  - **Specifity** Fraction of all negative instances that are incorrectly predicted positive.
     \begin{equation}\label{eq:specifity}
        \text{Specifity} = \text{FPR} = \frac{FP}{TN + FP}\\
     \end{equation}
     
  - **F-Score** Adjust $\beta$ for trade off between  precision and recall. For precision oriented task $\beta = 0.5$.
     \begin{equation}\label{eq:Fscore}
        F_\beta = \frac{(1+\beta^2) TP}{ (1+\beta^2) TP + \beta^2 FN + FP}
     \end{equation}



In [40]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [41]:
def DTW_prune(ts1, ts2):
    d,_ = dtw.warping_paths(ts1, ts2, window=10, use_pruning=True);
    return d

In [42]:
parameters = {'C':[5, 10, 13, 14, 15, 16, 17, 20, 40, 80],
              'kernel':['linear', 'poly', 'rbf', 'sigmoid'] # 'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'
              } # , 
SVM_classifier_balanced_00 = GridSearchCV(SVC(random_state=0, class_weight='balanced'), 
                                          parameters, cv=5, verbose=1)

SVM_classifier_balanced_00.fit(x_train_df.iloc[:, 1:], y_train_df.Vote.values)

print (SVM_classifier_balanced_00.best_params_)
print (SVM_classifier_balanced_00.best_score_)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
{'C': 10, 'kernel': 'rbf'}
0.968324277331015


In [43]:
parameters = {'C':[5, 10, 13, 14, 15, 16, 17, 20, 40, 80],
              'kernel':['linear', 'poly', 'rbf', 'sigmoid'] # 'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'
              } # , 
SVM_classifier_NoneWeight_00 = GridSearchCV(SVC(random_state=0), 
                                            parameters, cv=5, verbose=1)

SVM_classifier_NoneWeight_00.fit(x_train_df.iloc[:, 1:], y_train_df.Vote.values)

print (SVM_classifier_NoneWeight_00.best_params_)
print (SVM_classifier_NoneWeight_00.best_score_)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
{'C': 13, 'kernel': 'rbf'}
0.9646033470984567


In [44]:
SVM_classifier_NoneWeight_00_predictions = SVM_classifier_NoneWeight_00.predict(x_test_df.iloc[:, 1:])
SVM_classifier_balanced_00_predictions = SVM_classifier_balanced_00.predict(x_test_df.iloc[:, 1:])

In [45]:
balanced_y_test_df=y_test_df.copy()
None_y_test_df=y_test_df.copy()
balanced_y_test_df["prediction"] = list(SVM_classifier_balanced_00_predictions)
balanced_y_test_df.head(2)

,ID,Vote,prediction
1221,7667_WSDA_SF_2016,1,1
1334,99748_WSDA_SF_2017,1,1


In [46]:
None_y_test_df=y_test_df.copy()
None_y_test_df=y_test_df.copy()
None_y_test_df["prediction"] = list(SVM_classifier_NoneWeight_00_predictions)
None_y_test_df.head(2)

,ID,Vote,prediction
1221,7667_WSDA_SF_2016,1,1
1334,99748_WSDA_SF_2017,1,1


In [47]:
# Balanced Confusion Table

true_single_predicted_single=0
true_single_predicted_double=0

true_double_predicted_single=0
true_double_predicted_double=0

for index in balanced_y_test_df.index:
    curr_vote=list(balanced_y_test_df[balanced_y_test_df.index==index].Vote)[0]
    curr_predict=list(balanced_y_test_df[balanced_y_test_df.index==index].prediction)[0]
    if curr_vote==curr_predict:
        if curr_vote==1: 
            true_single_predicted_single+=1
        else:
            true_double_predicted_double+=1
    else:
        if curr_vote==1:
            true_single_predicted_double+=1
        else:
            true_double_predicted_single+=1
            
balanced_confus_tbl_test = pd.DataFrame(columns=['None', 'Predict_Single', 'Predict_Double'], 
                               index=range(2))
balanced_confus_tbl_test.loc[0, 'None'] = 'Actual_Single'
balanced_confus_tbl_test.loc[1, 'None'] = 'Actual_Double'
balanced_confus_tbl_test['Predict_Single']=0
balanced_confus_tbl_test['Predict_Double']=0

balanced_confus_tbl_test.loc[0, "Predict_Single"]=true_single_predicted_single
balanced_confus_tbl_test.loc[0, "Predict_Double"]=true_single_predicted_double
balanced_confus_tbl_test.loc[1, "Predict_Single"]=true_double_predicted_single
balanced_confus_tbl_test.loc[1, "Predict_Double"]=true_double_predicted_double
balanced_confus_tbl_test

,None,Predict_Single,Predict_Double
0,Actual_Single,214,5
1,Actual_Double,4,46


In [48]:
# None Weight Confusion Table

true_single_predicted_single=0
true_single_predicted_double=0

true_double_predicted_single=0
true_double_predicted_double=0

for index in None_y_test_df.index:
    curr_vote=list(None_y_test_df[None_y_test_df.index==index].Vote)[0]
    curr_predict=list(None_y_test_df[None_y_test_df.index==index].prediction)[0]
    if curr_vote==curr_predict:
        if curr_vote==1: 
            true_single_predicted_single+=1
        else:
            true_double_predicted_double+=1
    else:
        if curr_vote==1:
            true_single_predicted_double+=1
        else:
            true_double_predicted_single+=1
            
None_confus_tbl_test = pd.DataFrame(columns=['None', 'Predict_Single', 'Predict_Double'], 
                                    index=range(2))
None_confus_tbl_test.loc[0, 'None'] = 'Actual_Single'
None_confus_tbl_test.loc[1, 'None'] = 'Actual_Double'
None_confus_tbl_test['Predict_Single']=0
None_confus_tbl_test['Predict_Double']=0

None_confus_tbl_test.loc[0, "Predict_Single"]=true_single_predicted_single
None_confus_tbl_test.loc[0, "Predict_Double"]=true_single_predicted_double
None_confus_tbl_test.loc[1, "Predict_Single"]=true_double_predicted_single
None_confus_tbl_test.loc[1, "Predict_Double"]=true_double_predicted_double
None_confus_tbl_test

,None,Predict_Single,Predict_Double
0,Actual_Single,214,5
1,Actual_Double,6,44


In [49]:
print(classification_report(y_test_df.Vote, SVM_classifier_NoneWeight_00_predictions))

              precision    recall  f1-score   support

           1       0.97      0.98      0.97       219
           2       0.90      0.88      0.89        50

    accuracy                           0.96       269
   macro avg       0.94      0.93      0.93       269
weighted avg       0.96      0.96      0.96       269



In [50]:
print(classification_report(y_test_df.Vote, SVM_classifier_balanced_00_predictions))

              precision    recall  f1-score   support

           1       0.98      0.98      0.98       219
           2       0.90      0.92      0.91        50

    accuracy                           0.97       269
   macro avg       0.94      0.95      0.95       269
weighted avg       0.97      0.97      0.97       269



In [54]:
import pickle
model_dir = "/Users/hn/Documents/01_research_data/NASA/ML_Models/"
filename = model_dir + "SVM_classifier_balanced_regular"+ VI_idx +"_00.sav"
pickle.dump(SVM_classifier_balanced_00, open(filename, 'wb'))
filename

'/Users/hn/Documents/01_research_data/NASA/ML_Models/SVM_classifier_balanced_regularNDVI_00.sav'

In [55]:
filename = model_dir + "SVM_classifier_NoneWeight_regular"+ VI_idx +"_00.sav"
pickle.dump(SVM_classifier_NoneWeight_00, open(filename, 'wb'))
filename

'/Users/hn/Documents/01_research_data/NASA/ML_Models/SVM_classifier_NoneWeight_regularNDVI_00.sav'

In [63]:
None_y_test_df_act_1_pred_2=None_y_test_df[None_y_test_df.Vote==1].copy()
None_y_test_df_act_2_pred_1=None_y_test_df[None_y_test_df.Vote==2].copy()

None_y_test_df_act_1_pred_2=None_y_test_df_act_1_pred_2[None_y_test_df_act_1_pred_2.prediction==2].copy()
None_y_test_df_act_2_pred_1=None_y_test_df_act_2_pred_1[None_y_test_df_act_2_pred_1.prediction==1].copy()

None_y_test_df_act_2_pred_1

,ID,Vote,prediction
974,53244_WSDA_SF_2018,2,1
699,2660_WSDA_SF_2016,2,1
174,103257_WSDA_SF_2018,2,1
991,53718_WSDA_SF_2016,2,1
349,106509_WSDA_SF_2017,2,1
371,106968_WSDA_SF_2018,2,1


In [64]:
None_y_test_df_act_2_pred_1 = pd.merge(None_y_test_df_act_2_pred_1, 
                                       ground_truth_labels_extended, 
                                       on=['ID'], how='left')

None_y_test_df_act_1_pred_2 = pd.merge(None_y_test_df_act_1_pred_2, 
                                       ground_truth_labels_extended,
                                       on=['ID'], how='left')

print (None_y_test_df_act_2_pred_1.ExctAcr.sum()-None_y_test_df_act_1_pred_2.ExctAcr.sum())

418.46854102377984


In [65]:
balanced_y_test_df_act_1_pred_2=balanced_y_test_df[balanced_y_test_df.Vote==1].copy()
balanced_y_test_df_act_2_pred_1=balanced_y_test_df[balanced_y_test_df.Vote==2].copy()

balanced_y_test_df_act_1_pred_2=balanced_y_test_df_act_1_pred_2[balanced_y_test_df_act_1_pred_2.prediction==2].copy()
balanced_y_test_df_act_2_pred_1=balanced_y_test_df_act_2_pred_1[balanced_y_test_df_act_2_pred_1.prediction==1].copy()

balanced_y_test_df_act_2_pred_1

,ID,Vote,prediction
699,2660_WSDA_SF_2016,2,1
174,103257_WSDA_SF_2018,2,1
991,53718_WSDA_SF_2016,2,1
349,106509_WSDA_SF_2017,2,1


In [66]:
balanced_y_test_df_act_2_pred_1 = pd.merge(balanced_y_test_df_act_2_pred_1, 
                                       ground_truth_labels_extended, 
                                       on=['ID'], how='left')

balanced_y_test_df_act_1_pred_2 = pd.merge(balanced_y_test_df_act_1_pred_2, 
                                       ground_truth_labels_extended,
                                       on=['ID'], how='left')

print (balanced_y_test_df_act_2_pred_1.ExctAcr.sum()-balanced_y_test_df_act_1_pred_2.ExctAcr.sum())

253.92474911433752


# Balanced is the winner

In [69]:
print ((balanced_y_test_df_act_2_pred_1.ExctAcr.sum()-balanced_y_test_df_act_1_pred_2.ExctAcr.sum()).round(1))

253.9


In [70]:
balanced_confus_tbl_test

,None,Predict_Single,Predict_Double
0,Actual_Single,214,5
1,Actual_Double,4,46


In [71]:
balanced_y_test_df_act_2_pred_1.ExctAcr.sum()

485.4933915002344

In [72]:
balanced_y_test_df_act_1_pred_2.ExctAcr.sum()

231.56864238589688

In [ ]:
parameters = {'C':[5, 10, 13, 14, 15, 16, 17, 20, 40, 80],
              'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
              'class_weight':['balanced', None]} # , 
SVM_classifier_gridWeight = GridSearchCV(SVC(random_state=0), 
                                            parameters, cv=5, verbose=1,
                                            error_score='raise')

SVM_classifier_gridWeight.fit(x_train_df.iloc[:, 1:], y_train_df.Vote.values)

print (SVM_classifier_gridWeight.best_params_)
print (SVM_classifier_gridWeight.best_score_)